In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))

## 1. File Processing and Layout Matching

In [2]:
from typing import Text, Optional, Generator, List
import os, re, sys, json, pprint

def iter_files_in_dir(path: Text, ending: Optional[Text] = None) \
        -> Generator[Text, None, None]:
    for file_name in sorted_alphanumeric(os.listdir(path=path)):
        if ending:
            if file_name.endswith(ending):
                yield file_name.split('.')[0]
        else:
            yield file_name.split('.')[0]

def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    return sorted(data, key=alphanum_key)

def load_json_file(path):
    with open(path, 'r', encoding='utf8') as file:
        return json.load(file)
    
def split_bounds_in_lists(bounds):
    grouped_bounds = []
    curr_bounds = []
    for index, bound in enumerate(bounds):
        if index % 4 == 0:
            if curr_bounds:
                grouped_bounds.append(curr_bounds)
                curr_bounds = []
            curr_bounds.append(bound)
        else:
            curr_bounds.append(bound)
    grouped_bounds.append(curr_bounds)
    return grouped_bounds

def extract_layouts_comb(data):
    root = data['activity']['root']
    results = []
    queue = [root]
    while queue:
        elem = queue.pop()
        # Create result
        if elem:
            result = {'bounds': elem['bounds'], 'class': elem['class'], 'componentLabel': 'Layout'}
            results.append(result)
            # Extend ui comps
            if elem.get('children', None):
                for child in elem['children']:
                    queue.append(child)
    return results

def filter_comb_data(comb_data):
    all_bounds = set([(0, 0, 1440, 2560),(0, 0, 0, 0)])
    filtered_data = []
    for elem in comb_data:
        if tuple(elem.get("bounds")) not in all_bounds:
            all_bounds.add(tuple(elem.get("bounds")))
            filtered_data.append(elem)
    return filtered_data

def match_bounding_boxes(box_a, box_b):
    return (box_a[0] <= box_b[0]) and (box_a[1] <= box_b[1]) \
            and (box_a[2] >= box_b[2]) and (box_a[3] >= box_b[3])

def intersecting_bounding_boxes(box_a, box_b):
    return (box_a[0] <= box_b[0]) and (box_a[1] <= box_b[1]) \
            and (box_a[2] >= box_b[2]) and (box_a[3] >= box_b[3])

def match_layouts(layouts, ui_comps):
    matched_layouts = {}
    for elem in layouts:
        for ui_comp in ui_comps:
            if not (tuple(elem.get('bounds')) == tuple(ui_comp.get('bounds'))) and \
                    match_bounding_boxes(elem.get('bounds'), ui_comp.get('bounds')):
                if tuple(elem.get('bounds')) in matched_layouts:
                    matched_layouts[tuple(elem.get('bounds'))]['children'].append(ui_comp)
                else:
                    matched_layouts[tuple(elem.get('bounds'))] = {'bounds': elem.get('bounds'), 
                        'class': elem.get('class'), 'componentLabel': elem.get('componentLabel'), 
                        'children': [ui_comp]}
    return matched_layouts

def extract_images(results, image):
    images = []
    factor = 0.75
    for result in results:
        images.append(im.crop([int(x)*factor for x in result[0]]))
    return images

def get_all_file_indexes(path_base):
    dataset = pd.read_csv(PATH_BASE + 'preproc_text/dataset_new_stem_F_stpw_T_filter_T.csv')
    return [elem[0].split('.')[0] for elem in dataset[['filename']].values.tolist()]

def flatten_ui_comps_sema(path):
    json_sema = load_json_file(path)
    result = []
    ui_comps_init = json_sema['children']
    while ui_comps_init:
        ui_comp = ui_comps_init.pop()
        if ui_comp.get('children'):
            ui_comps_init.extend(ui_comp.get('children'))
        else:
            result.append(ui_comp)
    return result
    
layout_comp_labels = ['List Item', 'Toolbar', 'Card', 'Modal', 'Multi-Tab']   

## 2. GUI Component Data Extraction

In [3]:
MAX_SCREEN_WIDTH = 1440
MAX_SCREEN_HEIGHT = 2560
import pprint

def expand_layout(layout, layout_comp_labels):
    # Expand each comp that has children (=layout) and add it to the result set
    result = [layout]
    queue = [layout]
    while queue:
        curr_elem = queue.pop()
        for elem in curr_elem['children']:
            if elem['componentLabel'] in layout_comp_labels:
                result.append(elem)
                queue.append(elem)
    return result
    
def extract_data_for_gui(path_comb, path_sema, layout_comp_labels, top_filter_thresh=0.80, min_ui_comp_height=20):
    json_comb = load_json_file(path_comb)
    json_sema = load_json_file(path_sema)
    # Extract data from files
    comb_data = extract_layouts_comb(json_comb)
    filtered_comb_data = filter_comb_data(comb_data)
    # First use the basic sema ui comp data to match layouts against
    ui_comps = json_sema['children']
    matched_layouts = match_layouts(filtered_comb_data, ui_comps)
    # Filter all layouts with only one ui component or top_filter_thresh percentage
    # of ui components in the layout to reduce the number of non-meaningful layouts
    filtered_layouts = [v for k, v in matched_layouts.items() if  (len(v['children'])/len(ui_comps) < top_filter_thresh) \
                        and len(v['children']) > 1]
    # Add all layouts that are already in sema data to the overall layout list
    for ui_comp in ui_comps:
        if ui_comp['componentLabel'] in layout_comp_labels:
            filtered_layouts.extend(expand_layout(ui_comp, layout_comp_labels))
    # Compute the flattened ui comps from the sema data
    flattened_ui_comps = flatten_ui_comps_sema(path_sema)
    flattened_ui_comps = [ui_comp for ui_comp in flattened_ui_comps 
                          if (ui_comp['bounds'][3]-ui_comp['bounds'][1])>min_ui_comp_height]
    # Compute a minified version of the filtered layouts to only contain basic information
    minified_layouts = [{'componentLabel': e['componentLabel'], 'bounds':e['bounds'], 'class':e['class']}
                            for e in filtered_layouts]
    minified_layouts = [elem[0] for elem in sorted([(elem, (elem['bounds'][2]-elem['bounds'][0])*(elem['bounds'][3]-elem['bounds'][1]))
                            for elem in minified_layouts], key= lambda x: x[1], reverse=True)]
    return {'ui_comps': flattened_ui_comps, 'ui_comp_groups': filtered_layouts, 'min_ui_comp_groups':minified_layouts}

## 3. Image Style Property Extraction

In [5]:
def compute_bg_color(im_crop):
    # Computes the background color of an image crop as the main color in the histogram
    return sorted(im_crop.getcolors(im_crop.size[0]*im_crop.size[1]), key=lambda x: x[0], reverse=True)[0][1]

def compute_color_dist_lca(rgb_1, rgb_2):
    # Computes the color difference between two given colors based
    # on the real perceived difference of colors from humans instead of
    # using only the basic simple euclidean difference on the RGB values
    r_mean = (rgb_1[0] + rgb_2[0]) / 2
    diff_r = rgb_1[0] - rgb_2[0]
    diff_g = rgb_1[1] - rgb_2[1]
    diff_b = rgb_1[1] - rgb_2[1]
    return math.sqrt((2 + (r_mean / 256)) * diff_r**2 + 
                     4*diff_g**2 + (2 + ((255 - r_mean) / 256)) * diff_b**2)

In [6]:
import textdistance

def extract_text_color(im_crop, main_color, dist_thres = 0.85):
    try:
        # Compute the histogram for the image crop and sort it ascending based on prevelance
        colors = sorted(im_crop.getcolors(im_crop.size[0]*im_crop.size[1]), key=lambda x: x[0], reverse=True)
        # Background color is defined as the top color in the sorted histogram
        bg_color = colors[0][1]
        # Compute distance to all colors in the list to the background color with lca distance
        cols_with_dists = [(compute_color_dist_lca(bg_color, col[1]), col[0], col[1]) for col in colors]
        max_dist = int(max([col[0] for col in cols_with_dists])) | 1
        # Normalize the distances by the maximum distance in the data
        cols_with_norm_dists = [((color[0]/max_dist), color[1], color[2]) for color in cols_with_dists]
        # Find the 
        curr_max = -1
        curr_max_col = (0,0,0)
        for color in cols_with_norm_dists:
            if curr_max < color[0]:
                curr_max = color[0]
                curr_max_col = color[2]
            if color[0] >= dist_thres:
                return color[2]
        return curr_max_col
    except SystemError:
        # If the text color could not be calculated, then we return either black or
        # white depending on the max distance to the main background color
        RGB_WHITE (255,255,255)
        RGB_BLACK = (0,0,0)
        dist_white = compute_color_dist_lca(RGB_WHITE, main_color)
        dist_black = compute_color_dist_lca(RGB_BLACK, main_color)
        RGB_BLACK if dist_black > dist_white else RGB_WHITE
        
    
def extract_text_bb(img, detected_text, bounds, factor, text_ori, thresh=0.6):
    try:
        sim_text = detected_text.replace('\n', ' ').strip()
        sim = textdistance.damerau_levenshtein.normalized_similarity(text_ori.lower(), sim_text.lower())
        if detected_text and sim > thresh:
            d = pytesseract.image_to_boxes(img, output_type=Output.DICT)
            max_right = max(d['right'])
            min_bottom = min(d['bottom'])
            width, height = img.size
            right = bounds[0] + int(max_right)*(1/factor) if max_right > 0 else bounds[2]
            bottom = bounds[1] + (height - int(min_bottom))*(1/factor) if min_bottom > 0 else bounds[3]
            updated_bounds = [bounds[0]+int(d["left"][0])*(1/factor), bounds[1]+(height-int(d["top"][0]))*(1/factor), 
                    right, bottom]
            return updated_bounds
        return bounds
    except SystemError:
        return bounds

def extract_updated_text(img, detected_text, text, thresh=0.6):
    sim_text = detected_text.replace('\n', ' ').strip()
    sim = textdistance.damerau_levenshtein.normalized_similarity(text.lower(), sim_text.lower())
    if detected_text and sim > thresh:
        return detected_text.strip()
    else:
        return text

## 4. Main Script

In [7]:
from PIL import Image
import pandas as pd
import progressbar
from colormap import rgb2hex
from IPython.display import clear_output

import pytesseract
import cv2
import math
from pytesseract import Output

pytesseract.pytesseract.tesseract_cmd = r'\Tesseract-OCR\tesseract'

PATH_BASE = '../webapp/gui2rapp/staticfiles/resources/'
PATH_COMB = 'combined/'
PATH_SEMA = 'semantic_annotations/'

PATH_UI_COMPS = 'ui_comps_test/'
PATH_UI_COMPS_TEXT = 'ui_comps_text/'
PATH_COMB_EXTENDED = 'combined_extended_test/'

layout_comp_labels = ['List Item', 'Toolbar', 'Card', 'Modal', 'Multi-Tab']   

num_of_exceptions = 0

def iterate_and_load_dataset(path_base: Text, path_comb: Text, path_sema: Text, 
                             path_ui_comps: Text, path_ui_comps_text: Text, 
                             path_comb_extended: Text, layout_comp_labels, 
                             filter_comps: Optional[List[Text]] = [],
                             indexes = None):
    global num_of_exceptions
    if indexes:
        gen_all_file_indexes = indexes
    else:
        gen_all_file_indexes = get_all_file_indexes(path_base)
    dataframe = pd.DataFrame(columns=['id', 'componentLabel'])
    for file_index in progressbar.progressbar(gen_all_file_indexes):
        comps_data = extract_data_for_gui(path_base + path_comb + file_index + '.json',
                                         path_base + path_sema + file_index + '.json',
                                         layout_comp_labels)
        ui_comps = comps_data['ui_comps']
        ui_comp_groups = comps_data['ui_comp_groups']
        ui_comp_groups_min = comps_data['min_ui_comp_groups']
        all_comps = ui_comps.copy()
        all_comps.extend(ui_comp_groups)
        filtered_all_comps = [ui_comp for ui_comp in all_comps if not ui_comp.get('componentLabel') in filter_comps \
                            and ui_comp['bounds'][0] <= MAX_SCREEN_WIDTH and ui_comp['bounds'][1] <= MAX_SCREEN_HEIGHT \
                            and ui_comp['bounds'][2] <= MAX_SCREEN_WIDTH and ui_comp['bounds'][3] <= MAX_SCREEN_HEIGHT]
        img = Image.open(path_base + path_comb + file_index + '.jpg')
        width, height = img.size
        REF_HEIGHT = 2560
        factor = height / REF_HEIGHT
        # Extract the overall background color of the main part of the gui
        # and save it to the data so we can use it to create the background 
        # rectangle in edit mode with the correct color settings
        data_with_stp = {}
        bg_rect_crop = img.crop([int(x) * factor for x in [0, 84, 1440, 2392]])
        color_main = compute_bg_color(bg_rect_crop)
        hex_color = rgb2hex(*tuple(color_main))
        data_with_stp['bg_color'] = hex_color
        # Now we update all the ui comps or ui comp group with the style properties
        # Compute background color for the ui comp groups and add it to the dataset
        # Compute background for the screen cutout to fill the inital background rectangle
        for elem in ui_comp_groups_min:
            ui_comp_crop = img.crop([int(x) * factor for x in elem["bounds"]])
            color = compute_bg_color(ui_comp_crop)
            hex_color = rgb2hex(*tuple(color))
            elem['bg_color'] = hex_color
        # For each ui comp that is a text label, we compute additional style properties
        # that we add to the data in order to be able to correctly create the correct
        # label with the correct style properties in the edit mode later
        for elem in ui_comps:
            if elem['componentLabel'] == 'Text':
                # Extract the color of the text label
                ui_comp_crop = img.crop([int(x) * factor for x in elem["bounds"]])
                color = extract_text_color(ui_comp_crop, color_main)
                hex_color = rgb2hex(*tuple(color))
                elem['text_color'] = hex_color
                # Extract the updated bounding boxes of the text label
                ui_comp_crop_la = ui_comp_crop.convert('LA')
                detected_text = pytesseract.image_to_string(ui_comp_crop_la)
                elem['bounds_updated'] = extract_text_bb(ui_comp_crop_la, detected_text, elem['bounds'], factor, elem['text'])
                elem['text_updated'] = extract_updated_text(ui_comp_crop_la, detected_text, elem['text'])
            elif elem['componentLabel'] == 'Text Button':
                # Extract the background color of the text button
                ui_comp_crop = img.crop([int(x) * factor for x in elem["bounds"]])
                button_bg_color = compute_bg_color(ui_comp_crop)
                hex_color = rgb2hex(*tuple(button_bg_color))
                elem['bg_color'] = hex_color
                # Extract the text color of the text button
                color = extract_text_color(ui_comp_crop, button_bg_color)
                hex_color = rgb2hex(*tuple(color))
                elem['text_color'] = hex_color
                # Extract the updated bounding boxes of the text label
                ui_comp_crop_la = ui_comp_crop.convert('LA')
                detected_text = pytesseract.image_to_string(ui_comp_crop_la)
                button_text = elem.get('text') if elem.get('text') \
                              else (elem.get('textButtonClass') if elem.get('textButtonClass') else '')
                elem['bounds_text'] = extract_text_bb(ui_comp_crop_la, detected_text, elem['bounds'], factor, button_text)
                elem['text_updated'] = extract_updated_text(ui_comp_crop_la, detected_text, button_text)
            elif elem['componentLabel'] == 'Input' and 'text' in elem['class'].lower():
                # Extract the background color of the input field
                ui_comp_crop = img.crop([int(x) * factor for x in elem["bounds"]])
                input_bg_color = compute_bg_color(ui_comp_crop)
                hex_color = rgb2hex(*tuple(input_bg_color))
                elem['bg_color'] = hex_color
                # Extract the text color of the input field
                color = extract_text_color(ui_comp_crop, input_bg_color)
                hex_color = rgb2hex(*tuple(color))
                elem['text_color'] = hex_color
                # Extract the updated bounding boxes of the input field
                ui_comp_crop_la = ui_comp_crop.convert('LA')
                detected_text = pytesseract.image_to_string(ui_comp_crop_la)
                input_text = elem.get('text') if elem.get('text') else (detected_text if detected_text else '')
                elem['bounds_text'] = extract_text_bb(ui_comp_crop_la, detected_text, elem['bounds'], factor, input_text)
                elem['text_updated'] = extract_updated_text(ui_comp_crop_la, detected_text, input_text)
                # End input field extraction
        data_with_stp['ui_comps'] = ui_comps
        data_with_stp['ui_comp_groups'] = ui_comp_groups_min
        data_with_stp = draw_img_for_testing(img, data_with_stp)
        json.dump(data_with_stp, open(path_base + path_comb_extended + file_index + '.json', 'w' ), indent=4)
        clear_output()
    return data_with_stp

In [8]:
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox
import matplotlib


def draw_img_for_testing(im, data):
    # Display the image
    plt.figure(figsize=(20,10))
    plt.imshow(im)
    width, height = im.size
    # Get the current reference
    ax = plt.gca()
    factor = height / 2560
    print(factor)
    arr = [int(x) * factor for x in [0, 84, 1440, 2392]]
    bg_rect_crop = im.crop(arr)
    x = arr[0]
    y = arr[1]
    width = arr[2] - arr[0]
    height = arr[3] - arr[1]
    rect = Rectangle((x,y),width,height,linewidth=1,edgecolor='red',facecolor=data['bg_color'])
    ax.add_patch(rect)
    for ui_comp_group in data['ui_comp_groups']:
        a_bounds = [int(x)*factor for x in ui_comp_group['bounds']]
        x = a_bounds[0]
        y = a_bounds[1]
        width = a_bounds[2] - a_bounds[0]
        height = a_bounds[3] - a_bounds[1]
        rect = Rectangle((x,y),width,height,linewidth=1,edgecolor='gray',facecolor=ui_comp_group['bg_color'])
        ax.add_patch(rect)
    for result in data['ui_comps']:
        bound = result['bounds']
        a_bounds = [ int(x)*factor for x in bound ]
        x = a_bounds[0]
        y = a_bounds[1]
        width = a_bounds[2] - a_bounds[0]
        height = a_bounds[3] - a_bounds[1]
        ui_comp_crop = im.crop([int(x) * factor for x in result["bounds"]])
        # Create a Rectangle patch
        if result['componentLabel'] == 'Text':         
            # Draw new updated bounds rectangle
            bound = result['bounds_updated']
            a_bounds = [ int(x)*factor for x in bound ]
            x = a_bounds[0]
            y = a_bounds[1]
            width = a_bounds[2] - a_bounds[0]
            height = a_bounds[3] - a_bounds[1]
            rect = Rectangle((x,y),width,height,linewidth=1,edgecolor='red',facecolor='none')
            ax.add_patch(rect)
            
            bound2 = result['bounds']
            a2_bounds = [ int(x)*factor for x in bound2 ]
            x2= a2_bounds[0]
            y2 = a2_bounds[1]
            width2 = a2_bounds[2] - a2_bounds[0]
            height2 = a2_bounds[3] - a2_bounds[1]
            fontsize=int((height2/(0.1*len(result['text']))))
            fontsize2 = int(height2/4.5)
            
            font_size_easy = 12
            t = plt.text(x, y, result['text_updated'], size=font_size_easy, color=result['text_color'])
            plt.gcf().canvas.draw()
            bbox = t.get_window_extent().inverse_transformed(plt.gca().transData)
            t.set_visible(False)
            
            # Compute the difference of the widths of the orignal bounding box and the drawn for the given size
            w_ori = width
            w_new = bbox.x1 - bbox.x0
            w_diff = w_ori - w_new
            w_norm = w_diff / w_ori
            factor_font_size = 8 if factor > 0.5 else 11
            if result['bounds'] == result['bounds_updated']:
                new_font_size = 12
            else:
                new_font_size = font_size_easy + (factor_font_size * w_norm)
            plt.text(x, y+height, result['text_updated'], size=new_font_size, color=result['text_color'])
            result['font_size'] = new_font_size
            
            # Draw the original text bounds in blue
            bound = result['bounds']
            a_bounds = [int(x)*factor for x in bound ]
            x = a_bounds[0]
            y = a_bounds[1]
            width = a_bounds[2] - a_bounds[0]
            height = a_bounds[3] - a_bounds[1]
            rect = Rectangle((x,y),width,height,linewidth=1,edgecolor='blue',facecolor='none')
            ax.add_patch(rect)
        else:
            rect = Rectangle((x,y),width,height,linewidth=1,edgecolor='yellow',facecolor='none')
            ax.add_patch(rect)
    return data

In [ ]:
iterate_and_load_dataset(path_base=PATH_BASE, path_comb=PATH_COMB, path_sema=PATH_SEMA, 
                         path_ui_comps=PATH_UI_COMPS, path_ui_comps_text=PATH_UI_COMPS_TEXT, 
                         path_comb_extended=PATH_COMB_EXTENDED, layout_comp_labels=layout_comp_labels,
                         filter_comps=FILTER_COMPS)